In [3]:
# execution time tracking
!pip install ipython-autotime
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.0 MB/s eta 0:00:00
time: 197 µs (started: 2025-11-13 12:15:02 +00:00)


In [4]:
#libraries
!pip install transformers
!pip install torch
!pip install accelerate
!pip install bitsandbytes
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.5 MB/s eta 0:00:00
time: 23.3 s (started: 2025-11-13 12:15:06 +00:00)


In [5]:
#loading the mistral-7b-instruct model and the tokenizer
from torch import cuda, bfloat16
import transformers

model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth ='hf_token'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")
#tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    token=hf_auth
)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/configuration_auto.py:1317: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

time: 1min 37s (started: 2025-11-13 12:15:29 +00:00)


In [12]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.18 s (started: 2025-11-13 12:19:42 +00:00)


In [13]:
#loading test data for model evaluation
import pandas as pd
test_data_path='/content/drive/MyDrive/test_data_split.csv'

time: 417 µs (started: 2025-11-13 12:20:36 +00:00)


In [14]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

time: 562 µs (started: 2025-11-13 12:20:47 +00:00)


In [15]:
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

time: 734 µs (started: 2025-11-13 12:20:50 +00:00)


In [16]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

time: 808 µs (started: 2025-11-13 12:20:54 +00:00)


In [17]:
#processing the test data
input_csv = test_data_path
output_csv = '/content/drive/MyDrive/processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

Processed file saved as '/content/drive/MyDrive/processed_test_data.csv'.
time: 956 ms (started: 2025-11-13 12:22:10 +00:00)


In [18]:
#loading the processed test data
test_data_df=pd.read_csv('/content/drive/MyDrive/processed_test_data.csv')

time: 4.6 ms (started: 2025-11-13 12:22:18 +00:00)


In [19]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

time: 10.7 ms (started: 2025-11-13 12:22:20 +00:00)


In [20]:
#generating a lsit of prompts
prompts=[]
for a,b in zip(test_data_df['instructions'],test_data_df['texts']):
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Instruction: {a}

Input Text: {b}

[/INST]
'''
    prompts.append(prompt)

time: 1.01 ms (started: 2025-11-13 12:22:25 +00:00)


In [21]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

time: 270 µs (started: 2025-11-13 12:22:32 +00:00)


In [22]:
#verify the length of the prompt list
print(len(prompts))

130
time: 417 µs (started: 2025-11-13 12:22:38 +00:00)


In [23]:
#we now pass the list of prompts to the model to extract responses in an iterative manner.
#getting the output with transformers pipeline
#import pipeline
from transformers import pipeline
prompts=prompts
responses=[] #list to store the responses from the model.
for prompt in prompts:
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
  result = pipe(prompt)
  responses.append(result[0]['generated_text'][len(prompt):])

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 fo

time: 6min (started: 2025-11-13 12:22:43 +00:00)


In [26]:
#verify the responses
#print responses
responses

['The lasing frequency of the Quantum Cascade Semiconductor laser device is 3.3 THz.',
 'The optical power of the Quantum Cascade Semiconductor laser device is approximately 10 mW.',
 'The lasing frequency of the Quantum Cascade Semiconductor laser device is 2.07 THz.',
 'The working temperature of the Quantum Cascade Semiconductor laser device in pulsed mode is 190 K (or -87 degrees Celsius) and in continuous wave mode is 125 K (or -148 degrees Celsius).',
 'The working temperature of the Quantum Cascade Semiconductor laser device in continuous wave (cw) mode is 91 K.',
 'The heterostructure of the Quantum Cascade Semiconductor laser device, as described in the input text, is GaAs/Al0.23Ga0.77As.',
 'The heterostructure of the Quantum Cascade Semiconductor laser device, based on the provided input text, is In0.6Ga0.4As/InAs.',
 'The lasing frequency of the Quantum Cascade Semiconductor laser device is approximately 3.0 THz.',
 'The working temperature of the Quantum Cascade Semiconduc

time: 2.86 ms (started: 2025-11-13 12:30:15 +00:00)


In [25]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "mistral-7b-instruct-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to mistral-7b-instruct-responses.csv
time: 1.68 ms (started: 2025-11-13 12:28:43 +00:00)
